In [ ]:
# https://pub.towardsai.net/run-mixtral-8x7b-on-google-colab-free-d855686a385b

# https://www.youtube.com/watch?v=Zo3CTapKJ4I

# https://colab.research.google.com/github/dvmazur/mixtral-offloading/blob/master/notebooks/demo.ipynb

In [ ]:
pip install -q boto3 --progress-bar off

In [ ]:
# INSERT AWS KEYS AND TOKEN
from IPython.display import clear_output

my_aws_access_key_id = input("INSERT YOUR aws_access_key_id:")
my_aws_secret_access_key = input("INSERT YOUR aws_secret_access_key:")
my_aws_session_token = input("INSERT YOUR aws_session_token:")

clear_output()

In [ ]:
import boto3

# Inizializza il client DynamoDB con le credenziali manuali
dynamodb = boto3.resource(
    'dynamodb',
    region_name='eu-west-1',
    aws_access_key_id=my_aws_access_key_id,
    aws_secret_access_key=my_aws_secret_access_key,
    aws_session_token=my_aws_session_token
)

table = dynamodb.Table('processed_ids')

In [ ]:
# Funzione per aggiungere un nuovo record
def add_record(table, text_id):
    # Dati del nuovo record
    item = {
        'text_id': text_id  # Colonna e valore da inserire
        # Aggiungi qui altre colonne se necessario, ad esempio:
        # 'another_column': 'some_value'
    }

    # Aggiungi il record nella tabella
    try:
        table.put_item(Item=item)
        print(f"Record aggiunto con successo: {item}")
    except Exception as e:
        print(f"Errore durante l'inserimento del record: {e}")

# Funzione per estrarre tutti i valori della colonna 'text_id'
def extract_all_processed_text_ids(table):
    try:
        # Scansione iniziale della tabella
        response = table.scan()
        items = response['Items']

        # Lista per contenere i valori 'text_id'
        text_id_list = [item['text_id'] for item in items]

        # Gestione della paginazione (se ci sono più pagine di dati)
        while 'LastEvaluatedKey' in response:
            response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
            items = response['Items']
            text_id_list.extend([item['text_id'] for item in items])

        return text_id_list

    except Exception as e:
        print(f"Errore durante l'estrazione: {e}")
        return []

In [ ]:
import pickle

In [ ]:
experiment_name = "2024.09.23-02_AnalisiSentimentConMixtral"

In [ ]:
# !huggingface-cli login --token ####################

from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [ ]:
!huggingface-cli login --token $huggingface_cli_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"

input_data_path = base_path + "/../AnalisiTesti/data/output/full_data___6400_texts___sentiment_sdgwashing_sdgsmearing.xlsx"

output_data_path = base_path + "/../AnalisiTesti/data/output/" + experiment_name
log_filename = base_path + "/../AnalisiTesti/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
# @title
import pandas as pd
data = pd.read_excel(input_data_path)

In [ ]:
# @title
# import glob

# def update_data(data_sel):
#   drive.mount(mount_point, force_remount=True)

#   results = []
#   files = glob.glob(output_data_path + f"/*.pkl")
#   for file in files:
#     res = pd.read_pickle(file)
#     results.append(res)
#   print(len(results))

#   df = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'output'])
#   processed_texts = df.text_id.to_list()
#   print(processed_texts)

#   before = data_sel.shape[0]
#   print("Full dataset size:               ", before)
#   data = data_sel[~data_sel.text_id.isin(processed_texts)].copy() # DEEP COPY
#   after = data.shape[0]
#   print("Amount of files to be processed: ", after)
#   before - after

#   return data.sample(frac=1.00)

# # data_sel = update_data(data_sel)
# # data_sel.shape[0]

In [ ]:
# @title
# import glob

# def update_data(data_sel):
#   drive.mount(mount_point, force_remount=True)

#   processed_texts = []

#   files = glob.glob(output_data_path + f"/*.pkl")
#   for file in files:
#     processed_texts.append(file.split('_')[-1].split('.pkl')[0])

#   #print(processed_texts)

#   data = data_sel[~data_sel.text_id.isin(processed_texts)].copy() # DEEP COPY

#   print("Amount of processed files: ", len(processed_texts))

#   return data.sample(frac=1.00)

# # data_sel = update_data(data_sel)
# # data_sel.shape[0]

In [ ]:
import glob

def update_data(data_sel):

  processed_texts_from_dynamodb = extract_all_processed_text_ids(table)

  processed_texts_from_dynamodb = [int(x) for x in processed_texts_from_dynamodb]

  #print(processed_texts_from_dynamodb)

  data = data_sel[~data_sel.text_id.isin(processed_texts_from_dynamodb)].copy() # DEEP COPY

  print("Amount of processed files: ", len(processed_texts_from_dynamodb))

  return data.sample(frac=1.00)

# data_sel = update_data(data_sel)
# data_sel.shape[0]

# Mixtral in Colab

Welcome! In this notebook you can run [Mixtral8x7B-Instruct](https://huggingface.co/mistralai/Mixtral-8x7B-v0.1) with decent generation speed **right in Google Colab or on a consumer-grade GPU**. This was made possible by quantizing the original model in mixed precision and implementing a MoE-specific offloading strategy.

To learn more, read our [tech report](https://arxiv.org/abs/2312.17238) or check out the [repo](https://github.com/dvmazur/mixtral-offloading) on GitHub.

One will need approximately 16 GB of VRAM and 11 GB of RAM to run this notebook and generate somewhat long texts.


<details>

<summary>How to balance between RAM and GPU VRAM usage</summary>

You can balance between RAM and GPU VRAM usage by changing <code>offload_per_layer</code> variable in the <a href="#scrollTo=_mIpePTMFyRY&line=10&uniqifier=1">Initialize model</a> section. Increasing <code>offload_per_layer</code> will decrease GPU VRAM usage, increase RAM usage and decrease generation speed. Decreasing <code>offload_per_layer</code> will have the opposite effect.

Note that this notebook should run normally in Google Colab with <code>offload_per_layer = 4</code>, but may crush with other values. However, if you run this somewhere else, you're free to play with this variable.
</details>

## Install and import libraries

In [ ]:
# fix numpy in colab
import numpy
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

!git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
!cd mixtral-offloading && pip install -q -r requirements.txt
clear_output()

In [ ]:
!pip install --upgrade transformers
!pip install triton

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.1
    Uninstalling transformers-4.36.1:
      Successfully uninstalled transformers-4.36.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.9 MB/s eta 0:00:00


In [ ]:
import sys

sys.path.append("mixtral-offloading")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging

from src.build_model import OffloadConfig, QuantConfig, build_model

hf_logging.disable_progress_bar()

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


## Initialize model

In [ ]:
# !huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo >& mixtral_download.log

In [ ]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)
state_path = snapshot_download(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 4
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:453: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Loading experts:   0%|          | 0/32 [00:00<?, ?it/s]

## Run the model

In [ ]:
# from transformers import TextStreamer


tokenizer = AutoTokenizer.from_pretrained(model_name)
# streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
data_sel = data #.iloc[:3]

In [ ]:
data_sel = update_data(data_sel) # AGGIORNO IL DATAFRAME CON I DATI DA PROCESSARE A OGNI STEP

Amount of processed files:  5182


In [ ]:
results = []

while 1:

  data_sel = update_data(data_sel) # AGGIORNO IL DATAFRAME CON I DATI DA PROCESSARE A OGNI STEP

  if data_sel.shape[0]==0:
    break


  for index, row in data_sel.iterrows():

    cnt = index

    text_id = row['text_id']
    text = row["text"].replace('\n','').replace('{','(').replace('}',')')
    sdg_true = row["sdg_true"]

    file = open(log_filename, 'a+', encoding='utf-8')

    print("\n\n##################################################")
    file.write("\n\n##################################################\n")

    print("TEXT_ID:", text_id)
    print("TEXT:", row["text"].replace('\n',''),'\n\n')
    file.write(f"TEXT: " + row['text'].replace('\n','') + "\n\n\n")

  #   template = """[INST]
  # <<SYS>>
  # You are a helpful text classifier.
  # <</SYS>>
    template = """Given the following input text, between triple quotes, with its associated classification label with respect to one of the Sustainable Development Goals (SDGs), further classify the text with respect to the three labels defined below:

    - "positive": the text implies or explicitly states that what is affirmed or described leads to a significant advancement in favor of the goal indicated by the SDG under which it was previously classified.

    - "negative": the text implies or explicitly states that what is affirmed or described leads to a significant advancement against the goal indicated by the SDG under which it was previously classified.

    - "neutral": the text does not imply or explicitly state that what is affirmed or described leads to a significant advancement either in favor or against the goal indicated by the SDG under which it was previously classified.

  Return your answer in the following "list of strings" format

    [LABEL, LABEL_JUSTIFICATION_1, LABEL_JUSTIFICATION_2, LABEL_JUSTIFICATION_3, ...]

  where the first element must be the value of the label ("positive", "negative", or "neutral") and the subsequent values must contain the texts related to the features that justify the provided output label.

  Do NOT add any of your comments outside the list.

  The input text with its label are:
  '''
  INPUT TEXT: \""""+text+"""\"
  SDG_CLASSIFICATION: "SDG-"""+str(sdg_true)+"""\"
  OUTPUT:
  '''
  """
  # [/INST]
  # """

    # print(template)

    past_key_values = None
    sequence = None

    seq_len = 0

    #print("User: ", end="")
    #user_input = input()
    #print("\n")

    # user_entry = dict(role="user", content=user_input)
    user_entry = dict(role="user", content=template)
    input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

    if past_key_values is None:
      attention_mask = torch.ones_like(input_ids)
    else:
      seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
      attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=device)

    #print("Mixtral: ", end="")
    outputs = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      past_key_values=past_key_values,
      #streamer=streamer,
      do_sample=True,
      temperature=0.9,
      top_p=0.9,
      max_new_tokens=512,
      pad_token_id=tokenizer.eos_token_id,
      return_dict_in_generate=False, #True,
      output_hidden_states=False #, #True,
      # return_full_text = False
    )
    #print("\n")

    #sequence = result["sequences"]
    #past_key_values = result["past_key_values"]

    #print("result: ", tokenizer.decode(outputs[0]))
    #print(result.keys())
    #print(result['past_key_values'])
    #break
    prompt_length = input_ids.shape[1]
    result_extended = (cnt, text_id, sdg_true, tokenizer.decode(outputs[0][prompt_length:])) # AGGIUNGENDO [prompt_length:] SI ESCLUDE IL PROMPT DALL'OUTPUT!

    print(result_extended)
    file.write(str(result_extended)+'\n')

    with open(output_data_path+f'/sdg_classification_{text_id}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)

    add_record(table, str(text_id)) # AGGIUNGE L'ID DEL RECORD PROCESSATO NELLA TABELLA DI DYNAMODB, IN MODO CHE GLI ALTRI WORKER NON LO RIPROCESSINO

    file.close()

    break # PER FORZARE L'USCITA DAL CICLO INTERNO E PERMETTERE L'AGGIORNAMENTO DEL DATAFRAME

Amount of processed files:  5182


##################################################
TEXT_ID: 2921
TEXT: The Myanmar-China 2,520 km gas pipeline, with a capacity of 12 BCM per annum, became operational in October 2013. Furthermore, China started construction of the Eastern Russia-China gas pipeline for importing 5 BCM per annum from 2018 (the volume will increase to 30 BCM per annum from 2023). The length of the Chinese part of the pipeline is 3,170 km, which extends to Shanghai, and the Russian part is the 2,680 km Power of Siberia gas system connecting the Irkutsk region to Vladivostok. 




Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


(2659, 2921, 7, '["neutral",\n "The Myanmar-China gas pipeline becoming operational and having a capacity of 12 BCM per annum does not inherently indicate a significant advancement in SDG-7, as it depends on how the resources are used and the impact on energy access and efficiency.",\n "The China-Russia Eastern pipeline starting operation in 2018, initially importing 5 BCM per annum and increasing to 30 BCM per annum from 2023, does not directly indicate a significant advancement in SDG-7 without additional context about the energy sources and their impact on energy access, affordability, and efficiency.",\n "The length of the Chinese part of the pipeline (3,170 km) and the Russian part (2,680 km) does not provide information about SDG-7 advancement, as it depends on the type of energy being transported and its impact on energy access, affordability, and efficiency."]</s>')
Record aggiunto con successo: {'text_id': '2921'}
Amount of processed files:  5208


############################

In [ ]:
,STOP!!!

In [ ]:
data_sel = data #.iloc[:3]

In [ ]:
import glob

files = glob.glob(output_data_path + "/*.pkl")

results = []
for file in files:
    res = pd.read_pickle(file)

    # try:
    #     if len(res[3]['choices'])>1:
    #         raise Exception('ERROR: Multilabel detected!')
    #         break
    #     sdg_pred = res[3]['choices'][0]['message']['content']
    #     res4df = (res[0], res[1], res[2], sdg_pred)
    #     print(res4df)
    #     tuple_list.append(res4df)
    # except:
    #     print("#################################################################")
    #     print(res)
    #     print("#################################################################")

    # print(res)
    results.append(res)
    # break

len(results)

In [ ]:
results_processed = pd.DataFrame(results, columns=['cnt', 'text_id', 'sdg_true', 'result'])
results_processed.head(3)

In [ ]:
#results_processed.iloc[0].result.split("Answer with label only usign the format:---LABEL---")[1]
results_processed['result_short'] = results_processed['result'].apply(lambda x: x.split("Answer with label only usign the format:---LABEL---")[1])

In [ ]:
#results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed['label'] = results_processed['result_short'].str.extract(r'SDG (\d+)')
results_processed.head(3)

In [ ]:
final = data_sel.merge(results_processed, left_on='text_id', right_on='text_id')
final.head(3)

In [ ]:
text_ids_with_errors = final[final.label.isna()].text_id.to_list()
list(set(text_ids_with_errors))

In [ ]:
final  = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
# classification_report_filename = output_data_path+f'/{experiment_name}_classification_report.xlsx'
# classification_report_filename

In [ ]:
# report=classification_report(y_true, y_pred, output_dict=True)
# report_df = pd.DataFrame(report).transpose()
# report_df.to_excel(classification_report_filename)

In [ ]:
# test = pd.read_excel(classification_report_filename)
# test